#生成快视图

In [ ]:
import os
import numpy as np
from osgeo import gdal
from PIL import Image

# 配置参数
input_dir = r"D:\hupoxiazai\avg_46.7178_124.1012"   
output_dir = input_dir # 输出图像保存目录
red_band = 16 # 红色波段索引 (1-based)
green_band = 5 # 绿色波段索引
blue_band = 3 # 蓝色波段索引


# 创建输出目录
os.makedirs(output_dir, exist_ok=True)


def normalize_band(band_data):
# """归一化波段数据到0-255范围"""
    min_val = np.nanpercentile(band_data, 30)# 2%分位数
    max_val = np.nanpercentile(band_data, 82) # 98%分位数
    normalized = (band_data - min_val) / (max_val - min_val) * 255
    return np.clip(normalized, 0, 255).astype(np.uint8)


def process_tif(tif_path, output_path):
# """处理单个TIFF文件"""
    try:
    # 打开TIFF文件
        dataset = gdal.Open(tif_path)
        if not dataset:
            print(f"无法打开文件: {tif_path}")
            return

# 读取指定波段
        red = dataset.GetRasterBand(red_band).ReadAsArray()
        green = dataset.GetRasterBand(green_band).ReadAsArray()
        blue = dataset.GetRasterBand(blue_band).ReadAsArray()

        # 归一化波段
        red_norm = normalize_band(red)
        green_norm = normalize_band(green)
        blue_norm = normalize_band(blue)

        # 合并RGB通道
        rgb_array = np.dstack((red_norm, green_norm, blue_norm))

        # 创建并保存图像
        img = Image.fromarray(rgb_array)
        img.save(output_path)
        print(f"成功保存: {output_path}")

    except Exception as e:
        print(f"处理 {tif_path} 时出错: {str(e)}")
    

# 批量处理所有TIFF文件
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.tif')):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.png")
        process_tif(input_path, output_path)

print("所有文件处理完成！")